In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Seteo de Spark

In [4]:
!pip install pyspark py4j


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=2b566d1f4b26bd75ada24005e2584a3cab6ea210b21b40c6e2ae59785c89709e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("recorrido_bicicletas").getOrCreate()

# Funciones

In [7]:
# Cambia el nombre de multiples columnas reemplazando el valor viejo por el nuevo
# Recibe un diccionario donde la key es el nombre viejo de la columna y el value el nombre nuevo
def renombrar_columnas(df, renombres):
  for nombre_viejo, nombre_nuevo in renombres.items():
    df = df.withColumnRenamed(nombre_viejo, nombre_nuevo)
  return df

# Librerias

In [8]:
from pyspark.sql.functions import regexp_replace, concat_ws

# Estaciones

In [102]:
df_estaciones = spark.read.csv("/content/drive/MyDrive/Fudo Ejercicio/nuevas-estaciones-bicicletas-publicas.csv", \
            sep=';', \
            encoding='Windows-1252', \
            header='True', \
            inferSchema='True' \
          )
df_estaciones.show()


+--------+-------------------+-------------------+--------------------+----------------+---------+-------------+-------------------+------------------+
|ID Comet|NÚMERO de Estación |             NOMBRE|           DIRECCIÓN|          BARRIO|   COMUNA|EMPLAZAMIENTO|            LATITUD|          LONGITUD|
+--------+-------------------+-------------------+--------------------+----------------+---------+-------------+-------------------+------------------+
|       2|                  2|           RETIRO I|Av. Dr. José Marí...|          RETIRO| COMUNA 1|        PLAZA|-34.592424135701144|-58.37470989599143|
|       3|                  3|             ADUANA|            Moreno 4|       MONSERRAT| COMUNA 1|       VEREDA|           -34.6111|          -58.3682|
|       4|                  4|         PLAZA ROMA|  Av. Corrientes 100|     SAN NICOLAS| COMUNA 1|       VEREDA| -34.60299498843603|58.368832321694654|
|       5|                  5|       PLAZA ITALIA| Av. Sarmiento 2601 |         PALERMO|

In [103]:
renombres = {
    'ID Comet': 'id_comet',
    'NÚMERO de Estación ': 'nro_de_estacion',
    'NOMBRE': 'nombre',
    'DIRECCIÓN': 'direccion',
    'BARRIO': 'barrio',
    'COMUNA': 'comuna',
    'EMPLAZAMIENTO': 'emplazamiento',
    'LATITUD': 'latitud',
    'LONGITUD': 'longitud',
}

df_estaciones = renombrar_columnas(df_estaciones, renombres)


df_estaciones.show()

+--------+---------------+-------------------+--------------------+----------------+---------+-------------+-------------------+------------------+
|id_comet|nro_de_estacion|             nombre|           direccion|          barrio|   comuna|emplazamiento|            latitud|          longitud|
+--------+---------------+-------------------+--------------------+----------------+---------+-------------+-------------------+------------------+
|       2|              2|           RETIRO I|Av. Dr. José Marí...|          RETIRO| COMUNA 1|        PLAZA|-34.592424135701144|-58.37470989599143|
|       3|              3|             ADUANA|            Moreno 4|       MONSERRAT| COMUNA 1|       VEREDA|           -34.6111|          -58.3682|
|       4|              4|         PLAZA ROMA|  Av. Corrientes 100|     SAN NICOLAS| COMUNA 1|       VEREDA| -34.60299498843603|58.368832321694654|
|       5|              5|       PLAZA ITALIA| Av. Sarmiento 2601 |         PALERMO|COMUNA 14|       VEREDA|    

In [104]:
df_estaciones = df_estaciones.withColumn('comuna', regexp_replace('comuna', 'COMUNA ', ''))
df_estaciones.show()

+--------+---------------+-------------------+--------------------+----------------+------+-------------+-------------------+------------------+
|id_comet|nro_de_estacion|             nombre|           direccion|          barrio|comuna|emplazamiento|            latitud|          longitud|
+--------+---------------+-------------------+--------------------+----------------+------+-------------+-------------------+------------------+
|       2|              2|           RETIRO I|Av. Dr. José Marí...|          RETIRO|     1|        PLAZA|-34.592424135701144|-58.37470989599143|
|       3|              3|             ADUANA|            Moreno 4|       MONSERRAT|     1|       VEREDA|           -34.6111|          -58.3682|
|       4|              4|         PLAZA ROMA|  Av. Corrientes 100|     SAN NICOLAS|     1|       VEREDA| -34.60299498843603|58.368832321694654|
|       5|              5|       PLAZA ITALIA| Av. Sarmiento 2601 |         PALERMO|    14|       VEREDA|           -34.5807|     

# Usuarios

In [91]:
df_usuarios = spark.read.csv('/content/drive/MyDrive/Fudo Ejercicio/usuarios_ecobici_2023.csv', header = 'True', inferSchema='False')
df_usuarios.show()

+----------+--------------+------------+----------+---------+---------------------------+
|ID_usuario|genero_usuario|edad_usuario|fecha_alta|hora_alta|Customer.Has.Dni..Yes...No.|
+----------+--------------+------------+----------+---------+---------------------------+
|   1083476|          MALE|          20|2023-12-31| 19:24:57|                        Yes|
|   1083080|        FEMALE|          18|2023-12-31| 00:05:03|                        Yes|
|   1083163|         OTHER|          27|2023-12-31| 10:16:12|                         No|
|   1083435|          MALE|          24|2023-12-31| 17:42:11|                        Yes|
|   1083376|         OTHER|          32|2023-12-31| 16:16:15|                         No|
|   1083334|         OTHER|          34|2023-12-31| 15:07:05|                         No|
|   1083534|          MALE|          19|2023-12-31| 23:22:46|                        Yes|
|   1083473|          MALE|          37|2023-12-31| 19:16:00|                        Yes|
|   108353

In [92]:
renombramientos = {
    'ID_usuario': 'id_usuario',
    'genero_usuario': 'genero',
    'edad_usuario': 'edad',
    'Customer.Has.Dni..Yes...No.': 'posee_dni'
}

df_usuarios = renombrar_columnas(df_usuarios, renombramientos)

df_usuarios.show()

+----------+------+----+----------+---------+---------+
|id_usuario|genero|edad|fecha_alta|hora_alta|posee_dni|
+----------+------+----+----------+---------+---------+
|   1083476|  MALE|  20|2023-12-31| 19:24:57|      Yes|
|   1083080|FEMALE|  18|2023-12-31| 00:05:03|      Yes|
|   1083163| OTHER|  27|2023-12-31| 10:16:12|       No|
|   1083435|  MALE|  24|2023-12-31| 17:42:11|      Yes|
|   1083376| OTHER|  32|2023-12-31| 16:16:15|       No|
|   1083334| OTHER|  34|2023-12-31| 15:07:05|       No|
|   1083534|  MALE|  19|2023-12-31| 23:22:46|      Yes|
|   1083473|  MALE|  37|2023-12-31| 19:16:00|      Yes|
|   1083530| OTHER|  29|2023-12-31| 23:01:33|       No|
|   1083282|FEMALE|  24|2023-12-31| 13:23:39|      Yes|
|   1083442|FEMALE|  65|2023-12-31| 17:54:07|      Yes|
|   1083211|FEMALE|  18|2023-12-31| 11:37:33|      Yes|
|   1083204| OTHER|  24|2023-12-31| 11:28:13|       No|
|   1083436| OTHER|  20|2023-12-31| 17:44:33|       No|
|   1083418| OTHER|  25|2023-12-31| 17:06:32|   

In [93]:
df_usuarios = df_usuarios.withColumn('posee_dni', regexp_replace('posee_dni', 'Yes', 'Si'))

df_usuarios = df_usuarios.replace(['MALE', 'FEMALE', 'OTHER'], ['HOMBRE', 'MUJER', 'OTRO'], 'genero')
df_usuarios.show()

+----------+------+----+----------+---------+---------+
|id_usuario|genero|edad|fecha_alta|hora_alta|posee_dni|
+----------+------+----+----------+---------+---------+
|   1083476|HOMBRE|  20|2023-12-31| 19:24:57|       Si|
|   1083080| MUJER|  18|2023-12-31| 00:05:03|       Si|
|   1083163|  OTRO|  27|2023-12-31| 10:16:12|       No|
|   1083435|HOMBRE|  24|2023-12-31| 17:42:11|       Si|
|   1083376|  OTRO|  32|2023-12-31| 16:16:15|       No|
|   1083334|  OTRO|  34|2023-12-31| 15:07:05|       No|
|   1083534|HOMBRE|  19|2023-12-31| 23:22:46|       Si|
|   1083473|HOMBRE|  37|2023-12-31| 19:16:00|       Si|
|   1083530|  OTRO|  29|2023-12-31| 23:01:33|       No|
|   1083282| MUJER|  24|2023-12-31| 13:23:39|       Si|
|   1083442| MUJER|  65|2023-12-31| 17:54:07|       Si|
|   1083211| MUJER|  18|2023-12-31| 11:37:33|       Si|
|   1083204|  OTRO|  24|2023-12-31| 11:28:13|       No|
|   1083436|  OTRO|  20|2023-12-31| 17:44:33|       No|
|   1083418|  OTRO|  25|2023-12-31| 17:06:32|   

In [94]:
df_usuarios = df_usuarios.select('id_usuario', 'genero', 'edad', concat_ws(' ', df_usuarios.fecha_alta, df_usuarios.hora_alta).alias("fecha_de_alta"), 'posee_dni')

df_usuarios.show()

+----------+------+----+-------------------+---------+
|id_usuario|genero|edad|      fecha_de_alta|posee_dni|
+----------+------+----+-------------------+---------+
|   1083476|HOMBRE|  20|2023-12-31 19:24:57|       Si|
|   1083080| MUJER|  18|2023-12-31 00:05:03|       Si|
|   1083163|  OTRO|  27|2023-12-31 10:16:12|       No|
|   1083435|HOMBRE|  24|2023-12-31 17:42:11|       Si|
|   1083376|  OTRO|  32|2023-12-31 16:16:15|       No|
|   1083334|  OTRO|  34|2023-12-31 15:07:05|       No|
|   1083534|HOMBRE|  19|2023-12-31 23:22:46|       Si|
|   1083473|HOMBRE|  37|2023-12-31 19:16:00|       Si|
|   1083530|  OTRO|  29|2023-12-31 23:01:33|       No|
|   1083282| MUJER|  24|2023-12-31 13:23:39|       Si|
|   1083442| MUJER|  65|2023-12-31 17:54:07|       Si|
|   1083211| MUJER|  18|2023-12-31 11:37:33|       Si|
|   1083204|  OTRO|  24|2023-12-31 11:28:13|       No|
|   1083436|  OTRO|  20|2023-12-31 17:44:33|       No|
|   1083418|  OTRO|  25|2023-12-31 17:06:32|       No|
|   108309

In [95]:
df_usuarios = df_usuarios.withColumn('edad', regexp_replace('edad', ',\d*', ''))
df_usuarios = df_usuarios.withColumn('edad', regexp_replace('edad', ' ', ''))
df_usuarios.show()

+----------+------+----+-------------------+---------+
|id_usuario|genero|edad|      fecha_de_alta|posee_dni|
+----------+------+----+-------------------+---------+
|   1083476|HOMBRE|  20|2023-12-31 19:24:57|       Si|
|   1083080| MUJER|  18|2023-12-31 00:05:03|       Si|
|   1083163|  OTRO|  27|2023-12-31 10:16:12|       No|
|   1083435|HOMBRE|  24|2023-12-31 17:42:11|       Si|
|   1083376|  OTRO|  32|2023-12-31 16:16:15|       No|
|   1083334|  OTRO|  34|2023-12-31 15:07:05|       No|
|   1083534|HOMBRE|  19|2023-12-31 23:22:46|       Si|
|   1083473|HOMBRE|  37|2023-12-31 19:16:00|       Si|
|   1083530|  OTRO|  29|2023-12-31 23:01:33|       No|
|   1083282| MUJER|  24|2023-12-31 13:23:39|       Si|
|   1083442| MUJER|  65|2023-12-31 17:54:07|       Si|
|   1083211| MUJER|  18|2023-12-31 11:37:33|       Si|
|   1083204|  OTRO|  24|2023-12-31 11:28:13|       No|
|   1083436|  OTRO|  20|2023-12-31 17:44:33|       No|
|   1083418|  OTRO|  25|2023-12-31 17:06:32|       No|
|   108309

In [96]:
df_usuarios = df_usuarios.withColumn("edad",col("edad").cast("int"))
df_usuarios.show()

+----------+------+----+-------------------+---------+
|id_usuario|genero|edad|      fecha_de_alta|posee_dni|
+----------+------+----+-------------------+---------+
|   1083476|HOMBRE|  20|2023-12-31 19:24:57|       Si|
|   1083080| MUJER|  18|2023-12-31 00:05:03|       Si|
|   1083163|  OTRO|  27|2023-12-31 10:16:12|       No|
|   1083435|HOMBRE|  24|2023-12-31 17:42:11|       Si|
|   1083376|  OTRO|  32|2023-12-31 16:16:15|       No|
|   1083334|  OTRO|  34|2023-12-31 15:07:05|       No|
|   1083534|HOMBRE|  19|2023-12-31 23:22:46|       Si|
|   1083473|HOMBRE|  37|2023-12-31 19:16:00|       Si|
|   1083530|  OTRO|  29|2023-12-31 23:01:33|       No|
|   1083282| MUJER|  24|2023-12-31 13:23:39|       Si|
|   1083442| MUJER|  65|2023-12-31 17:54:07|       Si|
|   1083211| MUJER|  18|2023-12-31 11:37:33|       Si|
|   1083204|  OTRO|  24|2023-12-31 11:28:13|       No|
|   1083436|  OTRO|  20|2023-12-31 17:44:33|       No|
|   1083418|  OTRO|  25|2023-12-31 17:06:32|       No|
|   108309

In [100]:
df_usuarios = df_usuarios.filter((df_usuarios.edad >= 10) & (df_usuarios.edad <= 80))
df_usuarios.show()

+----------+------+----+-------------------+---------+
|id_usuario|genero|edad|      fecha_de_alta|posee_dni|
+----------+------+----+-------------------+---------+
|   1083476|HOMBRE|  20|2023-12-31 19:24:57|       Si|
|   1083080| MUJER|  18|2023-12-31 00:05:03|       Si|
|   1083163|  OTRO|  27|2023-12-31 10:16:12|       No|
|   1083435|HOMBRE|  24|2023-12-31 17:42:11|       Si|
|   1083376|  OTRO|  32|2023-12-31 16:16:15|       No|
|   1083334|  OTRO|  34|2023-12-31 15:07:05|       No|
|   1083534|HOMBRE|  19|2023-12-31 23:22:46|       Si|
|   1083473|HOMBRE|  37|2023-12-31 19:16:00|       Si|
|   1083530|  OTRO|  29|2023-12-31 23:01:33|       No|
|   1083282| MUJER|  24|2023-12-31 13:23:39|       Si|
|   1083442| MUJER|  65|2023-12-31 17:54:07|       Si|
|   1083211| MUJER|  18|2023-12-31 11:37:33|       Si|
|   1083204|  OTRO|  24|2023-12-31 11:28:13|       No|
|   1083436|  OTRO|  20|2023-12-31 17:44:33|       No|
|   1083418|  OTRO|  25|2023-12-31 17:06:32|       No|
|   108309

In [101]:
df_usuarios.select(min ('edad')).collect()

[Row(min(edad)=14)]

# Recorridos

No me alcanzo el tiempo para abrir correctamente el csv como un zip desde colab. Pero el procesamiento de este archivo seria equivalente a lo hecho en la resolucion con pandas y semejante a lo hecho para los otros 2 datasets.

Tambien falta crear las tablas y poblarlas, que es hacer con la libreria SQL de pyspark lo que ya se hizo con sqlite3.

In [12]:
#df_tours_tomados = pd.read_csv('/content/gdrive/MyDrive/Fudo Ejercicio/recorridos-realizados-2023.zip', compression='zip')
import zipfile
import io
# bzip2, gzip

#df_recorridos = spark.read.options(compression = 'bzip2', header=True).csv("/content/drive/MyDrive/Fudo Ejercicio/recorridos-realizados-2023.zip")

# Open the zip archive
with zipfile.ZipFile('/content/drive/MyDrive/Fudo Ejercicio/recorridos-realizados-2023.zip', 'r') as z:
  # Extract the file path of the CSV file within the zip archive
  print(f"z: {z}")
  csv_file_path = z.getinfo('/content/drive/MyDrive/Fudo Ejercicio/recorridos-realizados-2023.zip').filename

  # Read the CSV file into a PySpark DataFrame
  df_recorridos = spark.read.csv(csv_file_path, header=True, inferSchema=True)


#spark.read.options(delimiter=";", header=True).csv(path)

df_recorridos.show()

z: <zipfile.ZipFile filename='/content/drive/MyDrive/Fudo Ejercicio/recorridos-realizados-2023.zip' mode='r'>


KeyError: "There is no item named '/content/drive/MyDrive/Fudo Ejercicio/recorridos-realizados-2023.zip' in the archive"